<a href="https://colab.research.google.com/github/menosgrande/yahAuct-AVpricechecker/blob/main/yahAuct_AVpricechecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# 検索ワード入力
print('入力してください')
search_word = input().replace(" ","%20").replace("　","%E3%80%80")
# URLの指定        
url = 'https://erocate.com/search1/q-' + search_word + '/s-ya/c-ya_2084045794/?o=t1&vmode=1'
print(url)
#ユーザーエージェントの設定（設定必須）
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0"}
# ここでcookieを指定
cookie = {'adultauthed': '1617782919'}  #クッキーの指定
#htmlの取得
response = requests.get(url=url, headers=headers, cookies=cookie)
html = response.text
#BeautifulSoupで扱えるようにパースします
soup = BeautifulSoup(html,'lxml')
#URLタイトル
url_title = soup.title.string
print(url_title)
#全体平均
mean_price = soup.find(id = 'mean_price').text
mean_price_word_a = "平均:"
mean_price_word_b = "円"
mean_price_word = mean_price_word_a + str(mean_price) + mean_price_word_b
print(mean_price_word)


In [61]:
#右カラムから値段と作品タイトル取得
right_content = soup.find(id = 'right_contents')
#作品タイトルリスト

def list_up(list_name,category_name):

  for category_name in right_content.find_all(class_ = 'category_name'):
    category_name = category_name.text.replace("\u3000"," ").strip()
    list_name.append(category_name)

list_up(item_title_list,item_title)
list_up(item_price_list,item_price)

In [62]:
#商品リスト
test_list = [];
for i, p in enumerate(item_price_list):
    test_list.append({
        "価格": p,
        "タイトル": item_title_list[i],
    });

#表にする
df = pd.DataFrame(data = test_list)
css = {"background-color": "green"}
display(df.style.set_properties(subset="価格", **css))

,価格,タイトル
0,549 円,210307 向かいに越してきた隣人妻の絶倫セックスに溺れた僕 葵つかさ
1,290 円,初イキッ3本番！アイドル活動休止中の専属Gカップ人生で初めてイっちゃった！ 葵いぶき
2,600 円,T3 美人魔女MAX 2BIJN-179 高身長171cmミニスカ巨乳101cmGカップ白シャツ透けブラ長足美脚 SEXレス美人妻の欲求不満を満た～ 葵百合香
3,999 円,S1(7SSNI231)葵つかさ 執拗な焦らしと寸止めで極限まで感度を高めた恍惚キメセク大絶頂FUCK
4,400 円,巨乳痴女医に犯される逆セクハラ健康診断2 倉多まお 八乃つばさ 葵百合香 星空もあ
5,210 円,Ξ〓70 中古LEO〓ud888r∞8/絶対に声を出してはいけない状況なのに、カチコチにおっ起ったチ●ポを見せつけ 凛音とうか 葵百合香
6,371 円,Ξ〓29 中古センタービレッジ〓jrzd901∞初撮り人妻ドキュメント 池山葵
7,500 円,NSPS906/ながえ/布団の中のいけない性行為 交わっていけない男女がコソコソと戯れる密室空間/葵千恵/上原亜衣/167分
8,250 円,★新品★ゲッツ★性欲に飢えた美痴女がバイトする人妻デリヘル本番成功/葵百合香/伊東沙蘭/森下美緒/相浦茉莉花★GIRO-061
9,600 円,【アダルトDVD】『葵つかさ』ずっと憧れだった上司の妻つかささんと布団の中でこっそり密着セックスに没頭したボク (エスワン)


In [57]:
#平均値　LINE送信  
LINE_url = "https://notify-api.line.me/api/notify"
api_token = "wzq0M8MdrUrAqqGAcF5GYVDMdy1hM632xnOkaKQnfL4"
headers = {"Authorization": "Bearer " + api_token}

message = search_word +'は' + mean_price_word + '\n' + url
payload = {'message': message}
r = requests.post(LINE_url, headers=headers, params=payload)